## Task 1 - *Exploration of customer purchasing behavior*

### 1.1. Preprocessing

### *Importing liberaries*

In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
sys.path.append(os.path.abspath(os.path.join('../modules')))
warnings.filterwarnings("ignore")
sns.set()

In [49]:
from load_file import FileHandler
from overview import Df_Overview
from cleaner import DfCleaner
from outlier import DfOutlier
from vis_seaborn import VisSeaborn
from vis_plotly import VisPlotly

In [27]:
file_loader = FileHandler()

In [61]:
stored_df = file_loader.read_csv('../data/store.csv')
train_df = file_loader.read_csv('../data/train.csv')
test_df = file_loader.read_csv('../data/test.csv')

print("\n### General data Structure ###\n")

print("Stored data:",stored_df.shape)
print("Train data:",train_df.shape)
print("Test data:",test_df.shape)

2022-05-23 22:51:42,643 — DfHelper — DEBUG — file read as csv
2022-05-23 22:51:43,960 — DfHelper — DEBUG — file read as csv
2022-05-23 22:51:44,701 — DfHelper — DEBUG — file read as csv

### General data Structure ###

Stored data: (1115, 10)
Train data: (1017209, 9)
Test data: (41088, 8)


 ### *Feature Dicriptions of the Data* ###
 
The following are descriptions of **Rossmann Pharmaceuticals** data set.
- **`Id`** - an Id that represents a (Store, Date) duple within the test set
- **`Store`** - a unique Id for each store
- **`Sales`** - the turnover for any given day (this is what you are predicting)
- **`Customers`** - the number of customers on a given day
- **`Open`** - an indicator for whether the store was open: 0 = closed, 1 = open
- **`StateHoliday`** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. 

**Note that** all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- **`SchoolHoliday`** - indicates if the (Store, Date) was affected by the closure of public schools
- **`StoreType`** - differentiates between 4 different store models: a, b, c, d
- **`Assortment`** - describes an assortment level: a = basic, b = extra, c = extended. Read more about assortment here
- **`CompetitionDistance`** - distance in meters to the nearest competitor store
- **`CompetitionOpenSince[Month/Year]`** - gives the approximate year and month of the time the nearest competitor was opened
- **`Promo`** - indicates whether a store is running a promo on that day
- **`Promo2`** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- **`Promo2Since[Year/Week]`** - describes the year and calendar week when the store started participating in Promo2
- **`PromoInterval`** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store
- The data for this challenge can be found https://www.kaggle.com/competitions/rossmann-store-sales/data.

### Information Stored on each dataset

In [31]:
stored_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


## Data Preprocessing

### Cleaning

In [62]:
my_clean = DfCleaner()

In [54]:
store_overview = Df_Overview(stored_df)
store_overview.getOverview()

,count,none_count,none_percentage,unique_value_count,unique_percentage,dtype
label,,,,,,
Store,1115,0,0.0%,1115,100.0%,int64
StoreType,1115,0,0.0%,4,0.36%,object
Assortment,1115,0,0.0%,3,0.27%,object
Promo2,1115,0,0.0%,2,0.18%,int64
CompetitionDistance,1112,3,0.27%,654,58.65%,float64
CompetitionOpenSinceMonth,761,354,31.75%,12,1.08%,float64
CompetitionOpenSinceYear,761,354,31.75%,23,2.06%,float64
Promo2SinceWeek,571,544,48.79%,24,2.15%,float64
Promo2SinceYear,571,544,48.79%,7,0.63%,float64


**Observations:-**
- From the 10 columns, 6 have a null value. 
- The column with minimum missing values is in **CompetitionDistance.** and those can be filled with with a median   value.
- **PromoInterval, Promo2SinceYear, and Promo2SinceWeek** column has equal number of counts and missed values which is 544.
- **CompetitionOpenSinceYear, CompetitionOpenSinceMonth** column has equal number of counts and missed values which is .

In [63]:
stored_df = my_clean.fill_with_median(stored_df, ['CompetitionDistance'])

- looking in to `CompetitionDistance`

In [64]:
print(np.sort(stored_df['CompetitionOpenSinceMonth'].unique()))
print(np.sort(stored_df['CompetitionOpenSinceYear'].unique()))

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. nan]
[1900. 1961. 1990. 1994. 1995. 1998. 1999. 2000. 2001. 2002. 2003. 2004.
 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]


**Observation:-**
- `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear` Columns hold the year and month when a new competition opened.
- `CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear` having null value is meaning full. It means competition for a store has already been there before the store is established or there is no competition.
- The longest time we have in the dataset is 1900. So I decide to replaced the missed values with the first month 1990.

In [66]:
stored_df['CompetitionOpenSinceMonth'] = stored_df['CompetitionOpenSinceMonth'].fillna(1)
stored_df['CompetitionOpenSinceYear'] = stored_df['CompetitionOpenSinceYear'].fillna(1990)

- Looking in to the rest columns

In [67]:
print(np.sort(stored_df['Promo2SinceWeek'].unique()))
print(np.sort(stored_df['Promo2SinceYear'].unique()))
print(stored_df['PromoInterval'].unique())

[ 1.  5.  6.  9. 10. 13. 14. 18. 22. 23. 26. 27. 28. 31. 35. 36. 37. 39.
 40. 44. 45. 48. 49. 50. nan]
[2009. 2010. 2011. 2012. 2013. 2014. 2015.   nan]
[nan 'Jan,Apr,Jul,Oct' 'Feb,May,Aug,Nov' 'Mar,Jun,Sept,Dec']


In [68]:
stored_df[stored_df['Promo2'] == 0][['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].head(5)

,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


**Observation:-** 
- The three columns are dependent on Promo2, and those messing the value is because Promo2 is equal to **zero**. 
- So it is expected to create a new column holding if the store participated in promo2 and then we can replace Promo2SinceYear and Promo2SinceWeek with the earliest month and week.

In [70]:
print(pd.isnull(stored_df['PromoInterval'][0]))

True


In [ ]:
stored_df['Promo2SinceWeek'] = stored_df['Promo2SinceWeek'].fillna(0)
stored_df['Promo2SinceYear'] = stored_df['CompetitionOpenSinceYear'].fillna(0)

- From the previous resulst we observed that Entries in PromotionInterval have 4 entries. 
- So I am going to will split those points into 4 columns